# Utilizando Python e Google Spreadsheets para a criação de boletins escolares

## Introdução

A instituição de ensino na qual leciono utiliza há tempos o *Google Suite for Education* para a gestão das atividades escolares. Em nosso cotidiano, planilhas são as maiores aliadas, fazendo diversos papéis: quadros de informações, tabela de horários semanais, conteúdos das avaliações somativas, etc. O grande trunfo para nós, entretanto, está em sua utilização como **controle integrado de notas**, uma simplificação de um diário de classe tradicional, na qual lida-se apenas com as atividades realizadas e os desempenhos dos estudantes.

Ao longo dos últimos bimestres, tais controles vieram sendo progressivamente melhorados, acrescentando aquilo que nos parecia útil e retirando o que se percebeu não funcionar da maneira devida. No atual *estado da arte*, temos, dentre os principais recursos:

* uma 'planilha mestra', que lida com a tabulação de dados comuns a todas as disciplinas (tais como notas em atividades globais);
* uma 'planilha do professor', copiada via link a partir de um modelo, conectada à planilha mestra.

Tal padronização dos controles de notas nos permitiu adicionar outra função que se mostrava promissora: a geração de **boletins individuais** para cada estudante, especificando as atividades que foram realizadas em todas as disciplinas naquele período letivo, bem como o desempenho do aluno em cada uma, além dos valores finais — que já eram fornecidos em seu boletim escolar oficial.

Inicialmente, os boletins eram gerados a partir da extensão *autoCrat*, disponível na loja oficial do *Google Chrome* como um aplicativo para planilhas. Dentre suas funcionalidades, as que mais nos agradavam eram a possibilidade de gerar, para cada estudante, uma cópia de um 'modelo de boletim', populada com alguns de seus dados escolares, e enviá-la por e-mail, informando-lhe de que se tratava tal documento. Essa extensão ainda funciona e é por nós bastante apreciada.

Entretanto, no mês de outubro/2020, essa geração de boletins se mostrou bastante frustrante. O processo, que costumava ser de alguns segundos por boletim, passou a ser de mais de *dez minutos* para cada, gerando ainda por cima cópias falhas e sem os dados que necessitávamos. Próximos do início do período seguinte e das reuniões de pais e mestres, nas quais os boletins eram grandes aliados, nos vimos obrigados a buscar alternativas que atendessem às demandas da equipe docente.

Com base nisto iniciou-se uma nova abordagem, utilizando **python** e suas bibliotecas `gspread` e `pandas`.

## Índice

Nosso processo de geração dos boletins utilizou o seguinte roteiro:

1. Comunicação com a API e importação dos módulos
1. Importação dos dados a partir da planilha modelo e configuração do ambiente
1. Criação das cópias individuais e compartilhamento com estudantes e coordenação

Por questões razoáveis de privacidade, ao longo deste documento não exibiremos saídas reais dos dados recebidos, uma vez que se tratam de desempenhos reais de estudantes reais. Chaves para comunicação com a API do Google Drive e links para os modelos utilizados também serão ocultados. Tentaremos, sempre que possível, gerar saídas fantasia para explicar o que cada pedaço do script pretendia fazer.

## 1. Comunicação com a API e importação dos módulos

Para a utilização do módulo `gspread` é necessário realizar primeiramente uma autenticação, habilitando o acesso à API de um projeto no [Google Developers Code](https://console.developers.google.com/project). Selecionamos um projeto que já havia sido criado para testes — uma implementação incipiente de um sistema CRUD para gerenciamento de atividades escolares, ainda em fase de desenvolvimento — e copiamos a chave pública para um arquivo `creds.json`, salvo no mesmo diretório em que se localizava o script. Não é do escopo deste documento detalhar este processo, mas o leitor interessado pode conferir [alguns tutoriais](https://socraticowl.com/post/integrate-google-sheets-and-jupyter-notebooks/) que explicam de forma clara como criar [integrar suas planilhas google](https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html) com scripts python através da API do Google Drive.

Feito isto, pudemos iniciar nosso script importando os módulos `gspread` e `ServiceAccountCredentials`, como nos é [explicado na documentação](https://gspread.readthedocs.io/en/latest/oauth2.html):

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

Utilizamos nossas credenciais para nos conectarmos com a API, definindo os escopos necessários para interagir com planilhas Google e criar documentos no Drive:

In [ ]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
client = gspread.authorize(creds)

Importamos os módulos `pandas` e `logging` para lidar, respectivamente, com tratamento de dados e escrita dos registros. 

In [ ]:
import pandas as pd
import logging

O bloco a seguir faz uma rápida configuração do módulo `logging`, instanciando os objetos `logger` — que fará a escrita dos registros no arquivo `boletim.log` — e `console`, que direciona alguns desses registros para exibição no terminal. Essa etapa é de extrema importância para nosso projeto, principalmente por conta dos registros salvos em `boletim.log`. Caso ocorra algum erro no compartilhamento do boletim com algum estudante (processo que será detalhado mais à frente neste documento), poderemos verificar no registro o ID de tal documento, possibilitando assim um compartilhamento *manual*, se for o caso.

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%d-%b-%Y %H:%M:%S',
    filename='boletim.log',
    filemode='a'
)
# cria o objeto `logger`
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# handle que escreve mensagens tipo INFO em stderr
console = logging.StreamHandler()
console.setLevel(logging.INFO)

# formata exibição no console
formatter = logging.Formatter('%(levelname)-8s %(message)s')
console.setFormatter(formatter)

# adiciona handler `console` a `logger`
logger.addHandler(console)

## 2. Importação dos dados a partir da planilha modelo e configuração do ambiente

Quando fizemos o *design* da planilha de modelo do boletim, optamos por inserir uma página (escondida do usuário final) contendo todos os registros de todas as planilhas dos professores. Deste modo, as cópias individuais fazem uma busca nessa planilha `'Resumo'` por chaves que pertencem àquele estudante, exibindo assim somente seus dados.

Isto nos forneceu, nesta parte do projeto, um belo *dataset* dos desempenhos estudantis neste bimestre, a partir dos quais pudemos realizar a geração dos boletins individuais sem depender do complemento *autoCrat*. Caso seja de interesse da equipe docente, também poderemos num momento futuro analisar esses dados e procurar por informações relevantes, que possam dar *insights* e contrubuir com a atividade pedagógica.

Criamos, então, a variável `modeloID`, contendo o ID da planilha modelo e a utilizamos para abertura do documento:

In [ ]:
modeloID = '1HB...-38'  # truncada por fins de privacidade
modelo = client.open_by_key(modeloID)

Aberto o documento, selecionamos a planilha `'Resumo'` e puxamos todos os seus dados, através da função `get_all_values()`. Feito isto, transformamos este objeto num `DataFrame`. Para economizar requisições à API e facilitar a análise dos dados, utilizados a função `pandas.DataFrame.to_csv()` para salvar o dataframe no arquivo `resumo.csv`, recarregando novamente o objeto `resumo`.

In [ ]:
# abre planilha 'Resumo', puxa todos os seus valores e transforma em DataFrame
resumo = modelo.worksheet('Resumo').get_all_values()
columns = resumo[0]
resumo = pd.DataFrame(resumo[1:], columns=columns)

# salva em *.csv e recarrega `resumo`, agora a partir do arquivo
resumo.to_csv('resumo.csv', index=False)
resumo = pd.read_csv('resumo.csv')

# exibe informações de `resumo`
print(resumo.info())

```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Turma       2525 non-null   object
 1   Disciplina  2525 non-null   object
 2   n°          2525 non-null   int64
 3   Estudante   1235 non-null   object
 4   Bim         1190 non-null   object
 5   Con         2525 non-null   object
 6   Com         2525 non-null   object
 7   Dis         2525 non-null   object
 8   Média       2525 non-null   object
 9   Situação    2525 non-null   object
 10  C1          1195 non-null   object
 11  C2          1123 non-null   object
 12  C3          800 non-null    object
 13  C4          624 non-null    object
 14  C5          325 non-null    object
 15  C6          264 non-null    object
 16  C7          236 non-null    object
 17  C8          204 non-null    object
 18  C9          180 non-null    object
 19  C10         146 non-null    object
 20  C11         99 non-null     object
 21  C12         62 non-null     object
 22  C13         11 non-null     object
 23  C14         0 non-null      float64
 24  C15         0 non-null      float64
 25  C16         0 non-null      float64
 26  C17         0 non-null      float64
 27  C18         0 non-null      float64
 28  C19         0 non-null      float64
 29  C20         0 non-null      float64
dtypes: float64(7), int64(1), object(22)
```

O dataframe inicial continha 2526 linhas e 30 colunas. Na coluna `'Estudante'` vemos 1235 entradas não nulas, indicando portanto 1291 linhas com objetos `NaN`. Isto ocorre porque as planilhas dos professores, por definição, contém mais linhas do que o número de estudantes em cada turma — uma escolha de *design* que permite a matrícula de novos alunos durante o ano letivo, sem prejudicar a padronização dos documentos.

Removendo do dataframe essas linhas, nosso objeto `resumo` contém, agora, 1235 linhas e 30 colunas, a partir das quais podemos iniciar a configuração do boletim.

In [ ]:
# remove linhas de `resumo` sem estudantes
resumo = resumo.dropna(subset=['Estudante'])

Para enviar os boletins por e-mail, criamos um dataframe a partir do arquivo `lista_emails.csv`, já gerado a partir de script utilizado em outros projetos, armazenando-o no objeto `lista_emails`.

In [ ]:
lista_emails = pd.read_csv('lista_emails.csv')

Utilizamos a função `pandas.Series.unique` para extrair os valores únicos da coluna `'Turma'` e armazená-los no objeto `turmas`. Feito isto, pudemos iniciar o processo de geração de cópias do modelo e compartilhamento com estudantes e coordenação

In [ ]:
turmas = resumo.Turma.unique()

## 3. Criação das cópias individuais e compartilhamento com estudantes e coordenação

Iteramos através do objeto `turmas` e, para cada `turma`, iteramos através dos números de chamada dos estudantes, no intervalo de 1 a 25. O esqueleto do *loop*, portanto, ficou como segue:

In [ ]:
# itera através das turmas
for turma in turmas:
    # itera através dos estudantes
    for numero in range(1, 26):
        pass

Para cada estudante, fizemos um *subset* de nosso dataframe, localizando apenas as linhas nas quais `turma` e `numero` correspondem, respectivamente, às colunas `'Turma'` e `'n°'`, armazenando-o no objeto `notas`:

In [ ]:
notas = resumo.loc[(resumo['Turma'] == turma) & (resumo['n°'] == numero)]

Fizemos uma busca pelo nome do estudante no dataframe `lista_emails` para receber seu endereço de e-mail. Como estamos iterando entre os números 1 e 25, chegaremos num ponto do laço no qual não há mais estudantes. Para não gerar erros, tal verificação e feita dentro de um bloco `try` que, encontrando uma exceção, continua a execução do laço para o próximo item.

In [ ]:
try:
    # nome e email do estudante
    estudante = notas.iloc[0,3]
    email = lista_emails.loc[(lista_emails['Nome'] == estudante)]
    email = email.iloc[0,2]

except:
    continue

De posse desses dados, pudemos enfim criar uma cópia da planilha modelo para o estudante, através da função `copy()`. Essas chamadas também vieram dentro de um bloco `try` para lidar com possíveis exceções, além de registrar em `logger` mensagens de sucesso e erro.

In [ ]:
# cria uma cópia de `modelo` com o título `titulo`
titulo = f'{turma} _ {numero}. {estudante} (boletim 4bim-2020)'
try:
    copia = client.copy(modeloID, title=titulo, copy_permissions=True)
    logger.info(f'Criou cópia para {turma} {estudante}')
    logger.info(f'ID: {copia.id}')
except:
    logger.error(f'Não foi possivel criar cópia para {estudante}')
    continue

Criada a cópia, alteramos as colunas `'B1'` e `'B2'`, inserindo respectivamente a turma e o número do estudante. O restante da *magia* fica por conta da planilha, que filtra os dados da página `'Resumo'` com as informações das colunas acima mencionadas.

In [ ]:
# atualiza células com turma e número
boletim = copia.worksheet('Boletim')
boletim.update('B1', turma)
boletim.update('B2', numero)

Finalizada a edição, compartilhamos o documento com os endereços de e-mail do estudante, da coordenação e do *rapaz do TI*. O primeiro tem permissão apenas de leitura, para evitar que altere seus dados — podendo inclusive visualizar informações de outros estudantes — enquanto que os dois últimos têm permissão de edição.

In [ ]:
# email a ser enviado para o estudante
message = f'''EMAIL AUTOMÁTICO, NÃO RESPONDA!

Olá, {estudante}, tudo bem?

Compartilho neste e-mail seu boletim individual de notas do 3º bimestre de 2020. Nele, você pode conferir como foi seu desempenho em todas as atividades deste bimestre, em todas as disciplinas. Assim, fica mais fácil de saber quais instrumentos seus professores utilizaram para te avaliar, o valor de cada item e a nota que você obteve.

Muito importante: isso serve apenas para você conferir suas notas e ver se está tudo certo. Caso encontre algum problema, peço que entre em contato com o(a) professor(a) responsável por aquela matéria e converse diretamente com ele(a), combinado? Às vezes pode acontecer de nós, professores, errarmos também! Mas vou te dar uma dica: primeiro, confira se você realmente entregou aquela atividade, via PLURALL, antes de questionar o(a) professor(a), pode ser? E caso a nota deste relatório esteja diferente do seu boletim oficial, emitido via Notas Online, comunique o(a) professor(a) também — geralmente é apenas uma "diferença de arredondamentos", que não prejudica em nada sua nota!

Qualquer dúvida, pode falar comigo (prof. Pedro), com a coordenação ou enviar um e-mail para o suporte: s...@esc...br.

Até já! (:

EMAIL AUTOMÁTICO, NÃO RESPONDA!'''

# permissões: coordenação, TI e estudante
try:
    copia.share(email, perm_type='user', role='reader', notify=True, email_message=message)
    copia.share('d...@esc...br', perm_type='user', role='writer')
    copia.share('p...@esc...br', perm_type='user', role='writer', notify=True, email_message=message)
except:
    logger.error('Não foi possível compartilhar o documento.')

Chegamos, assim, ao fim do processo! Foram gerados 98 boletins, de 8 turmas diferentes, compartilhados com os estudantes via e-mail e notificação. Para alguns deles, não recebemos uma cópia por e-mail, por razões que fogem de nossa compreensão. Contudo, e mais importantes, todos eles estavam na seção *Compartilhados comigo* do Google Drive, com as permissões devidas e corretamente sinalizadas.

Foi um trabalho de algumas horas, com indas e vindas. Talvez pudéssemos ter levado o mesmo (ou menos) tempo gerando cópias *na unha*. Porém, além de enfadonha e propensa a erros, tal metodologia não nos leva a **aprender** nada. E, como educadores, acreditamos que a aprendizagem deve vir em primeiro lugar e estimulada sempre que possível.

### Autor

Pedro P. Bittencourt, [professor da educação básica](http://pedrobittencourt.com.br/curriculum) e [programador aspirante](https://github.com/pbittencourt).